# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


### Questions 1 & 2

In [2]:
sum(square_root_generator(5))

8.382332347441762

In [3]:
num = 13
[
    value for i, value in enumerate(square_root_generator(num)) if i == num - 1
][0]

3.605551275463989

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [4]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [5]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

### Questions 3 & 4

In [6]:
import dlt
import duckdb

In [7]:
pipeline_name = 'hw'
!rm {pipeline_name}.duckdb

pipeline = dlt.pipeline(pipeline_name=pipeline_name, destination='duckdb')
duck_con = duckdb.connect(f'{pipeline_name}.duckdb')

rm: cannot remove 'hw.duckdb': No such file or directory


In [8]:
appended_table_name = 'people_appended'

first_ingestion_info = pipeline.run(people_1(), table_name=appended_table_name, primary_key='ID', write_disposition='append')
second_ingestion_info = pipeline.run(people_2(), table_name=appended_table_name, primary_key='ID', write_disposition='append')
print(first_ingestion_info)
print(second_ingestion_info)

Pipeline hw load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/hw.duckdb location to store data
Load package 1707343824.207236 is LOADED and contains no failed jobs
Pipeline hw load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/hw.duckdb location to store data
Load package 1707343824.9468977 is LOADED and contains no failed jobs


In [9]:
duck_con.sql(f'SELECT * FROM people.{appended_table_name}')

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707343824.207236  │ 4/g1LbrEt5iBYg │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707343824.207236  │ rBvNennV9LaIoA │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1707343824.207236  │ mL+rLK+gUBUvJQ │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1707343824.207236  │ e46r7kZNQx+PDw │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1707343824.207236  │ TCrOlBVGc2AZ6w │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1707343824.9468977 │ E3XAoiCnq1KwUA │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1707343824.9468977 │ SPzotkyKANFwZA │ Job_4      │
│     5 │ 

In [10]:
duck_con.sql(f'SELECT SUM(age) FROM people.{appended_table_name}')

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

In [11]:
merged_table_name = 'people_merged'

first_ingestion_info = pipeline.run(people_1(), table_name=merged_table_name, primary_key='ID', write_disposition='merge')
second_ingestion_info = pipeline.run(people_2(), table_name=merged_table_name, primary_key='ID', write_disposition='merge')
print(first_ingestion_info)
print(second_ingestion_info)

Pipeline hw load step completed in 0.43 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/hw.duckdb location to store data
Load package 1707343825.688407 is LOADED and contains no failed jobs
Pipeline hw load step completed in 0.44 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/hw.duckdb location to store data
Load package 1707343826.5512278 is LOADED and contains no failed jobs


In [12]:
duck_con.sql('SELECT * FROM people.people_merged ORDER BY name')

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707343825.688407  │ IBN1kMoTU3V/MA │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707343825.688407  │ sMvCGPYmONpefQ │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1707343826.5512278 │ bWdA6DFVcEc38Q │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1707343826.5512278 │ ccDFbhOlcEdozg │ Job_4      │
│     5 │ Person_5 │    35 │ City_B  │ 1707343826.5512278 │ VaoZp0WxyValyQ │ Job_5      │
│     6 │ Person_6 │    36 │ City_B  │ 1707343826.5512278 │ JSka8nKBB60KqQ │ Job_6      │
│     7 │ Person_7 │    37 │ City_B  │ 1707343826.5512278 │ H8o/bIWIAUGu2A │ Job_7      │
│     8 │ 

In [13]:
duck_con.sql('SELECT SUM(age) FROM people.people_merged')

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX